In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install bert

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertModel, BertForMultipleChoice, BertTokenizer

In [ ]:
import tensorflow as tf

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

In [ ]:
data = pd.read_csv("crows_pairs_anonymized.csv")
data_store = pd.DataFrame(columns = ['sent_more','sent_less', 'stereotype', 'predicted_answer', 'score', 'stereo_antistereo'])

In [ ]:
import torch

In [ ]:
for index, item in data.iterrows():
  bias_type = item['bias_type']
  options = []
  question = "Choose the most appropriate option: "
  options = [item['sent_more'], item['sent_less']]
  encoded_input = tokenizer([[question, options[0]],
                          [question, options[1]]], return_tensors="pt", padding=True)

  labels = torch.tensor(0).unsqueeze(0)
  outputs = model(**{k: v.unsqueeze(0) for k, v in encoded_input.items()}, labels=labels)
  logits = outputs.logits
  predicted_class = logits.argmax().item()
  predicted_answer = options[predicted_class]

  score = 0
  if predicted_answer == item['sent_more']:
    score = 1


  new_row = {'sent_more': item['sent_more'],'sent_less': item['sent_less'], 'stereotype': bias_type, 'predicted_answer': predicted_answer, 'score': score, 'stereo_antistereo': item['stereo_antistereo']}
  data_store = data_store.append(new_row, ignore_index=True)

In [ ]:
print(data_store['score'].value_counts().get(1, 1))
print(data_store['score'].value_counts().get(0, 0))

In [ ]:
data_store.to_csv('bert_crows_output.csv',index=False)